In [111]:
import pandas as pd
import matplotlib as plt
import math
import datetime

In [25]:
usersDF = pd.read_csv('../../../Downloads/myanimelist/users_cleaned.csv')
animesDF = pd.read_csv('../../../Downloads/myanimelist/anime_cleaned.csv')
scoresDF = pd.read_csv('../../../Downloads/myanimelist/scores.csv')
scores_sortedDF = pd.read_csv('../../../Downloads/myanimelist/scores_sorted.csv')

In [28]:
print(usersDF.shape, animesDF.shape, scoresDF.shape, scores_sortedDF.shape)

(108711, 17) (6668, 33) (19066725, 4) (19066725, 4)


In [26]:
scoresDF.head()

,username,anime_id,my_score,my_status
0,karthiga,21,9,1
1,karthiga,59,7,2
2,karthiga,74,7,2
3,karthiga,120,7,2
4,karthiga,178,7,2


In [27]:
scores_sortedDF.head()

,username,anime_id,my_score,my_status
0,bushwhacker2k,1,7,1
1,swampybr,1,9,2
2,Uotani,1,9,2
3,JoeAteMyPencil,1,10,2
4,XuXu89,1,8,1


In [129]:
def get_viewer(scoresDF, viewer):
    return scoresDF.loc[(scoresDF['username'] == viewer)]

print(datetime.datetime.now())
get_viewer(scoresDF, 'Faultydoombot')
print(datetime.datetime.now())

2020-01-31 18:17:34.924598
2020-01-31 18:17:36.371449


In [148]:
# change to numpy arrays??
def sim_pearson(scores, anime1_id, anime2_id):
    '''Return the Pearson correlation coefficient for anime1 and anime2.'''
    scores_anime1 = scores[scoresDF.anime_id == anime1_id]
    scores_anime2 = scores[scoresDF.anime_id == anime2_id]
    # get the list of shared viewers
    shared_viewers = 0
    scores1 = []
    scores2 = []
    count = 0
    n = scores_anime1.shape[0]
    for index, row in scores_anime1.iterrows():
        viewer = get_viewer(scores_anime2, row['username'])
        if not viewer.empty:
            shared_viewers += 1
            scores1.append(row['my_score'])
            scores2.append(int(viewer['my_score']))
        count += 1
        if count % 1000 == 0:
            print('Viewers cheked: {} out of {} :: Shared: {} :: {}'.format(count, n, shared_viewers, datetime.datetime.now()))
            
    # if no ratings in common, return 0
    if shared_viewers == 0:
        return 0

    # add up all the preferences
    sum1 = sum(scores1)
    sum2 = sum(scores2)
    
    # sum up the squares
    sum1Sq = sum([pow(score, 2) for score in scores1])
    sum2Sq = sum([pow(score, 2) for score in scores2])
    
    # sum up the products
    pSum = sum([score1*score2 for score1, score2 in zip(scores1, scores2)])
    
    # calculate Pearson score
    num = pSum - (sum1 * sum2)/shared_viewers
    den = sqrt((sum1Sq - pow(sum1, 2)/shared_viewers) * (sum2Sq - pow(sum2, 2)/shared_viewers))
    if den == 0:
        return 0
    else:
        return num/den # between(-1, 1)

In [149]:
sim_pearson(scoresDF, 1735, 20) #comparing Naruto and Naruto: Shippuuden

Viewers cheked: 1000 out of 39015 :: Shared: 940 :: 2020-01-31 18:49:45.469201
Viewers cheked: 2000 out of 39015 :: Shared: 1883 :: 2020-01-31 18:49:51.909977
Viewers cheked: 3000 out of 39015 :: Shared: 2811 :: 2020-01-31 18:49:57.800839
Viewers cheked: 4000 out of 39015 :: Shared: 3754 :: 2020-01-31 18:50:03.669125
Viewers cheked: 5000 out of 39015 :: Shared: 4701 :: 2020-01-31 18:50:09.593985
Viewers cheked: 6000 out of 39015 :: Shared: 5647 :: 2020-01-31 18:50:15.662693
Viewers cheked: 7000 out of 39015 :: Shared: 6577 :: 2020-01-31 18:50:21.517537
Viewers cheked: 8000 out of 39015 :: Shared: 7523 :: 2020-01-31 18:50:27.276128
Viewers cheked: 9000 out of 39015 :: Shared: 8477 :: 2020-01-31 18:50:33.024539
Viewers cheked: 10000 out of 39015 :: Shared: 9422 :: 2020-01-31 18:50:38.843304
Viewers cheked: 11000 out of 39015 :: Shared: 10368 :: 2020-01-31 18:50:44.677069
Viewers cheked: 12000 out of 39015 :: Shared: 11314 :: 2020-01-31 18:50:50.452456
Viewers cheked: 13000 out of 39015 :

0.7065833687115664